This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json"
params = dict(key=API_KEY, start_date='2017-01-02', end_date='2017-01-03')
res = requests.get(url, params = params)
new_json = res.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

## We obtain a dictionary with one key, dataset_data
## Our data 
print(new_json)

## Our data is stored as a list of lists, indexed by the key 'data'
## Each list is a row, corresponding to one day.
## Column names are stored in the list indexed by 'column_index'

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-01-02', 'end_date': '2017-01-03', 'frequency': 'daily', 'data': [['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
##Solution to Problem 1.

url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json"
## Set the `start_date' and `end_date' to collect data on all of 2017.
params = dict(key=API_KEY, start_date='2017-01-01', end_date='2017-12-31')

## The variable json `red`contains all data from 2017.
res = requests.get(url, params = params)

In [6]:
## Solution to Problem 2.

## We convert the json from Problem 1 to a dictionary.
data = res.json()
print(type(data))

<class 'dict'>


In [7]:
## We define some helper problems that we will use for the remaining problems.

def get_index(col_name):
    """Helper function that gives the index of a column name for our data set."""
    column_names = data['dataset_data']['column_names']
    return(column_names.index(col_name))

def get_col(col_name):
    """Returns the column with given name from our data set."""
    idx = get_index(col_name)
    return[row[idx] for row in data['dataset_data']['data']]

In [8]:
## Solution to Problem 3.

## Get list of opening numbers
opening_nums = get_col('Open')
## Remove missing values, which will cause the max and min functions to break
opening_nums = list(filter(lambda num: type(num) == float, opening_nums))
print("The highest opening price was: " + str(max(opening_nums)))
print("The lowest opening price was: " + str(min(opening_nums)))

The highest opening price was: 53.11
The lowest opening price was: 34.0


In [9]:
## Solution to Problem 4

## Get list of high prices for all days
high_prices = get_col('High')
## Get list of low prices for all days
low_prices = get_col('Low')
## Subtract low from high for each day
differences = [round(high - low, 2) for high, low in zip(high_prices, low_prices)]

print("The largest difference between high and low prices in any one day was: "
        + str(max(differences)))

The largest difference between high and low prices in any one day was: 2.81


In [10]:
## Solution to Problem 5
## We interpret the question to mean a change between two consecutive days. 
## We assume 'largest' means largest magnitude, so take absolute values of the differences.

closing_nums = get_col('Close')
## List absolute value of differences between consecutive days
diffs = [abs(closing_nums[idx] - closing_nums[idx+1]) 
         for idx in range(len(closing_nums)-1)]
max_diff = round(max(diffs), 2)
print("The largest difference between any two consecutive days had a magnitude of: " + str(max_diff))


The largest difference between any two consecutive days had a magnitude of: 2.56


In [11]:
## Solution to Problem 6

## Get list of all trading volumes
trading = get_col('Traded Volume')
## Compute average: sum all entries, and divide by the number of entries
avg = round(sum(trading)/len(trading), 2)

print("The average trading volume was: "+str(avg))

The average trading volume was: 89124.34
